In [1]:
!guardrails hub install hub://guardrails/lowercase --quiet
!guardrails hub install hub://guardrails/two_words --quiet
!guardrails hub install hub://guardrails/one_line --quiet

%pip install pypdfium2

Installing hub://guardrails/lowercase...
✅Successfully installed guardrails/lowercase!


Installing hub://guardrails/two_words...
✅Successfully installed guardrails/two_words!


Installing hub://guardrails/one_line...
✅Successfully installed guardrails/one_line!



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Using Guardrails with Chat Models

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/extracting_entities.ipynb).

In this example, we will set up Guardrails with a chat model.

## Objective

We retry the [entity extraction example](./extracting_entities.ipynb) using a chat model.

## Step 0: Download PDF and load it as string

To get started, download the document from [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/data/chase_card_agreement.pdf) and save it in `data/chase_card_agreement.pdf`.

Guardrails has some built-in functions to help with common tasks. Here, we will use the `read_pdf` function to load the PDF as a string.

In [2]:
import guardrails as gd
from rich import print

content = gd.docs_utils.read_pdf("data/chase_card_agreement.pdf")

print(f"Chase Credit Card Document:\n\n{content[:275]}\n...")

/Users/dtam/.pyenv/versions/3.12.1/lib/python3.12/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


Chase Credit Card Document:

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prim
...

## Step 1: Create the RAIL Spec with `<instructions>` tags

In order to use Guardrails with a chat model, we need to add `<instructions>` tags to the RAIL spec. These tags will be used to generate the system message for the chat model.

Ordinarily, everything that is contained in the `<prompt>` tag will be split across `<prompt>` and `<instructions>` tags. Here's an example illustrating the differences.


=== "RAIL Spec with instruction tag"

    ```xml
    <instructions>
    You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ${gr.json_suffix_prompt_examples}
    </instructions>


    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
    `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    ${gr.xml_prefix_prompt}

    ${output_schema}
    </prompt>
    ```

=== "Pydantic with instructions"
    ```py
    instructions = """You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ${gr.json_suffix_prompt_examples}"""

    prompt = """Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    ${gr.xml_prefix_prompt}

    ${output_schema}"""
    ```

=== "RAIL Spec without instruction tag"
    
    ```xml
    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    ${gr.xml_prefix_prompt}

    ${output_schema}

    ${gr.json_suffix_prompt_v2_wo_none}
    </prompt>
    ```

=== "Pydantic without instructions"
    ```py
    prompt = """Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    ${gr.xml_prefix_prompt}

    ${output_schema}

    ${gr.json_suffix_prompt_v2_wo_none}"""
    ```

After materialization, the two variations of the RAIL specification will look like this when passed to the LLM (regarless if they were initialized from xml or a Pydantic model):

=== "RAIL Spec with instruction tag"

    ```xml
    <instructions>
    You are a helpful assistant only capable of communicating with valid JSON, and no other text.

    ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

    Here are examples of simple (XML, JSON) pairs that show the expected behavior:
    - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
    - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
    - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`
    </instructions>


    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
    `null`.

    ${document}

    Extract information from this document and return a JSON that follows the correct schema.

    Given below is XML that describes the information to extract from this document and the tags to extract it into.

    ${output_schema}
    </prompt>
    ```

=== "RAIL Spec without instruction tag"

    ```xml
    <prompt>
    Given the following document, answer the following questions. If the answer doesn't exist in the document, enter `null`.

    ${document}

    Given below is XML that describes the information to extract from this document and the tags to extract it into.

    ${output_schema}

    ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

    Here are examples of simple (XML, JSON) pairs that show the expected behavior:
    - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
    - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
    - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`
    </prompt>
    ```

Here's the final RAIL spec as XML:

In [3]:
rail_str = """
<rail version="0.1">

<output>

    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="hub://guardrails/one_line; hub://guardrails/two_words" on-fail-lower-case="fix" on-fail-two-words="reask" />
            <string name="explanation" format="hub://guardrails/one_line" />
            <float name="value" format="percentage"/>
        </object>
    </list>
    <object name="interest_rates" required="false" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?" />
</output>


<instructions>
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

${gr.json_suffix_prompt_examples}
</instructions>


<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
`null`.

${document}

Extract information from this document and return a JSON that follows the correct schema.

${gr.xml_prefix_prompt}

${output_schema}
</prompt>

</rail>
"""

Or using a Pydantic model for the output schema:

In [4]:
from guardrails.hub import LowerCase, TwoWords, OneLine
from pydantic import BaseModel, Field

prompt = """
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

${document}

${gr.complete_xml_suffix_v2}
"""

class Fee(BaseModel):
    name: str = Field(validators=[LowerCase(on_fail="fix"), TwoWords(on_fail="reask")])
    explanation: str = Field(validators=[OneLine()])
    value: float = Field(description="The fee amount in USD or as a percentage.")

class AccountFee(BaseModel):
    account_type: str = Field(validators=[LowerCase(on_fail="fix")])
    rate: float = Field(description="The annual percentage rate (APR) for the account type.")

class CreditCardAgreement(BaseModel):
    fees: list[Fee] = Field(description="What fees and charges are associated with my account?")
    interest_rates: list[AccountFee] = Field(description="What are the interest rates offered by the bank on different kinds of accounts and products?")

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

Creating the guard from XML:

In [5]:
guard = gd.Guard.from_rail_string(rail_str)

Or from the Pydantic model:

In [6]:
guard = gd.Guard.from_pydantic(output_class=CreditCardAgreement)

As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

In [9]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

raw_llm_response, validated_response, *rest = guard(
    model="gpt-4o",
    prompt=prompt,
    prompt_params={"document": content[:6000]},
    max_tokens=2048,
    temperature=0,
)

We see in the prompt that was sent to the LLM, the `{document}` is substituted with the user provided value at runtime.

In [10]:
print(guard.history.last.compiled_prompt)

Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
'None'.

2/25/23, 7:59 PM about:blank
about:blank 1/4
PRICING INFORMATION
INTEREST RATES AND INTEREST CHARGES
Purchase Annual
Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prime
Rate.
a
My Chase Loan
SM APR 19.49%. This APR will vary with the market based on the Prime Rate.
a
Promotional offers with fixed APRs and varying durations may be available from
time to time on some accounts.
Balance Transfer APR 0% Intro APR for the first 18 months that your Account is open.
After that, 19.49%. This APR will vary with the market based on the Prime
Rate.
a
Cash Advance APR 29.49%. This APR will vary with the market based on the Prime Rate.
b
Penalty APR and When
It Applies
Up to 29.99%. This APR will vary with the market based on the Prime Rate.
c
We may apply the Penalty APR to your account if you:
fail to make a Minimum Payment by the date and time that it is due; or
make a payment to us that is returned unpaid.
How Long Will the Penalty APR Apply?: If we apply the Penalty APR for
either of these reasons, the Penalty APR could potentially remain in effect
indefinitely.
How to Avoid Paying
Interest on Purchases
Your due date will be a minimum of 21 days after the close of each billing cycle.
We will not charge you interest on new purchases if you pay your entire balance
or Interest Saving Balance by the due date each month. We will begin charging
interest on balance transfers and cash advances on the transaction date.
Minimum Interest
Charge
None
Credit Card Tips from
the Consumer Financial
Protection Bureau
To learn more about factors to consider when applying for or using a credit card,
visit the website of the Consumer Financial Protection Bureau at
http://www.consumerfinance.gov/learnmore.
FEES
Annual Membership
Fee
None
My Chase Plan
SM Fee
(fixed finance charge)
Monthly fee of 0% of the amount of each eligible purchase transaction or
amount selected to create a My Chase Plan while in the 0% Intro Purchase
APR period.
After that, monthly fee of 1.72% of the amount of each eligible purchase
transaction or amount selected to create a My Chase Plan. The My Chase Plan
Fee will be determined at the time each My Chase Plan is created and will
remain the same until the My Chase Plan is paid in full.
d
Transaction Fees
Balance Transfers Intro fee of either $5 or 3% of the amount of each transfer, whichever is greater,
on transfers made within 60 days of account opening. After that: Either $5 or 5%
of the amount of each transfer, whichever is greater.
Cash Advances Either $10 or 5% of the amount of each transaction, whichever is greater.
2/25/23, 7:59 PM about:blank
about:blank 2/4
Foreign Transactions 3% of the amount of each transaction in U.S. dollars.
Penalty Fees
Late Payment Up to $40.
Over-the-Credit-Limit None
Return Payment Up to $40.
Return Check None
Note: This account may not be eligible for balance transfers.
Loss of Intro APR: We will end your introductory APR if any required Minimum Payment is 60 days late, and
apply the Penalty APR.
How We Will Calculate Your Balance: We use the daily balance method (including new transactions).
Prime Rate: Variable APRs are based on the 7.75% Prime Rate as of 2/7/2023.
aWe add 11.74% to the Prime Rate to determine the Purchase/My Chase Loan/Balance Transfer APR.
Maximum APR 29.99%.
bWe add 21.74% to the Prime Rate to determine the Cash Advance APR. Maximum APR 29.99%.
cWe add up to 26.99% to the Prime Rate to determine the Penalty APR. Maximum APR 29.99%.
dMy Chase Plan Fee: The My Chase Plan Fee is calculated at the time each plan is created and is based on
the amount of each purchase transaction or amount selected to create the plan, the number of billing periods
you choose to pay the balance in full, and other factors. The monthly and aggregate dollar amount of your M

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [11]:
print(validated_response)

{
    'fees': [
        {'name': 'annual membership', 'explanation': 'None', 'value': 0.0},
        {
            'name': 'my chase',
            'explanation': 'Monthly fee of 0% of the amount of each eligible purchase transaction or amount 
selected to create a My Chase Plan while in the 0% Intro Purchase APR period. After that, monthly fee of 1.72% of 
the amount of each eligible purchase transaction or amount selected to create a My Chase Plan.',
            'value': 0.0
        },
        {
            'name': 'balance transfers',
            'explanation': 'Intro fee of either $5 or 3% of the amount of each transfer, whichever is greater, on 
transfers made within 60 days of account opening. After that: Either $5 or 5% of the amount of each transfer, 
whichever is greater.',
            'value': 5.0
        },
        {
            'name': 'cash advances',
            'explanation': 'Either $10 or 5% of the amount of each transaction, whichever is greater.',
            'value': 10.0
        },
        {
            'name': 'foreign transactions',
            'explanation': '3% of the amount of each transaction in U.S. dollars.',
            'value': 3.0
        },
        {'name': 'late payment', 'explanation': 'Up to $40.', 'value': 40.0},
        {'name': 'return payment', 'explanation': 'Up to $40.', 'value': 40.0}
    ],
    'interest_rates': [
        {'account_type': 'purchase', 'rate': 0.0},
        {'account_type': 'purchase', 'rate': 19.49},
        {'account_type': 'my chase loan', 'rate': 19.49},
        {'account_type': 'balance transfer', 'rate': 0.0},
        {'account_type': 'balance transfer', 'rate': 19.49},
        {'account_type': 'cash advance', 'rate': 29.49},
        {'account_type': 'penalty', 'rate': 29.99}
    ]
}

In [12]:
guard.history.last.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following document, answer the following questions. If the answer doesn't exist in the        │ │
    │ │ document, enter 'None'.                                                                                 │ │
    │ │                                                                                                         │ │
    │ │ 2/25/23, 7:59 PM about:blank                                                                            │ │
    │ │ about:blank 1/4                                                                                         │ │
    │ │ PRICING INFORMATION                                                                                     │ │
    │ │ INTEREST RATES AND INTEREST CHARGES                                                                     │ │
    │ │ Purchase Annual                                                                                         │ │
    │ │ Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account is open.                   │ │
    │ │ After that, 19.49%. This APR will vary with the market based on the Prime                               │ │
    │ │ Rate.                                                                                                   │ │
    │ │ a                                                                                                       │ │
    │ │ My Chase Loan                                                                                           │ │
    │ │ SM APR 19.49%. This APR will vary with the market based on the Prime Rate.                              │ │
    │ │ a                                                                                                       │ │
    │ │ Promotional offers with fixed APRs and varying durations may be available from                          │ │
    │ │ time to time on some accounts.                                                                          │ │
    │ │ Balance Transfer APR 0% Intro APR for the first 18 months that your Account is open.                    │ │
    │ │ After that, 19.49%. This APR will vary with the market based on the Prime                               │ │
    │ │ Rate.                                                                                                   │ │
    │ │ a                                                                                                       │ │
    │ │ Cash Advance APR 29.49%. This APR will vary with the market based on the Prime Rate.                    │ │
    │ │ b                                                                                                       │ │
    │ │ Penalty APR and When                                                                                    │ │
    │ │ It Applies                                                                                              │ │
    │ │ Up to 29.99%. This APR will vary with the market based on the Prime Rate.                               │ │
    │ │ c                                                                                                       │ │
    │ │ We may apply the Penalty APR to your account if you:                                                    │ │
    │ │ fail to make a Minimum Payment by the date and time that it is due; or                                  │ │
    │ │ make a payment to us that is returned unpaid.                                                           │ │
    │ │ How Long Will the Penalty APR Apply?: If we apply the Penalty APR for                                   │ │
    │ │ either of these reasons, the Penalty APR co